Code for generating the basic template for the diagram of evolving dynamics in the paper (Figure 2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

# Directory where the plots are saved
dir_path = "../plots"
# Parameters
num_steps = 30
alpha = 0.5
tails = True    # plot also the trajectories of tokens as they evolve
colors = ["#56B4E9", "#009E73","#E69F00", "#CC79A7", "#0072B2", "#D55E00", "#F0E442",'C6', 'C7', 'C8', 'C9', 'C10']
s1 = 1 / (1 + alpha)
s2 = alpha * s1

## Define the different initial token values
cases = [1,0,2,3] #0 = emerging leader, 1 = triangular polytope, 2 = square polytope, 3 = non-leader clusters
saving_iter = [0, 1,2, 5, num_steps]
fig, axs = plt.subplots(np.size(cases), np.size(saving_iter), figsize=(5, 4.3))
# Adjust spacing
plt.subplots_adjust(wspace=0.1, hspace=0.2)
for idx, case in enumerate(cases):
    if case == 0:
        z0 = np.array([[1,4/12], [-1/12, 1/12], [-1,-1], [0, 3/12]]).T
        d, _ = z0.shape
        added_z = np.array([[-0.3, -0.7], [-0.6, -0.9]]).T
        added_z2 = np.array([[0.8, 0.1], [0.6, 0.15]]).T
        z0 = np.append(z0,added_z, axis = 1)
        z0 = np.append(z0,added_z2, axis = 1)
    elif case == 1:
        z0 = np.array([[1, 1], [0.5, -0.5], [-1, -1]]).T
        d, _ = z0.shape
        added_z = np.random.uniform(low=0, high=1, size=(d, 2))
        added_z2 = np.random.uniform(low=-1, high=0, size=(d, 2))
        added_z3 = np.array([[0.3, -0.2], [0.1, -0.2]]).T
        z0 = np.append(z0,added_z, axis = 1)
        z0 = np.append(z0,added_z2, axis = 1)
        z0 = np.append(z0,added_z3, axis = 1)
    elif case == 2:
        z0 = np.array([[1, 1],[1, -1] , [-1, -1], [-1, 1]]).T
        d, _ = z0.shape
        m = 6 #number of extra particles
        added_z = np.random.uniform(low=-0.5, high=0.5, size=(d, m))
        z0 = np.append(z0,added_z, axis = 1)
    else:
        z0 = np.array([[1, 1], [1, -1], [-1, -1], [-1, 1],[-0.5, 0], [0.5, 0]]).T
        d, _ = z0.shape
        m = 4 #number of extra particles
        added_z = np.random.uniform(low=-0.5, high=0.5, size=(d, m))
        z0 = np.append(z0,added_z, axis = 1)
    
    d, n = z0.shape
    A = np.identity(d)
    W = np.zeros((d, n))
    dt_string = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = dt_string + ".gif"
    base_filename = dt_string
    ## Run the HARDMAX dynamics ##
    z = np.zeros((d, n, num_steps+1)) #we want to do num_steps number of iterations, + 1 to save the initial configuration
    z[:, :, 0] = z0
    f = z0.copy()
    for iter in range(1,num_steps+1):
        # Get dynamics
        for i in range(n):
            IP = np.dot(f[:, i].T, f)
            Pij = np.zeros(n)
            ind = IP == np.max(IP)
            Pij[ind] = 1. / np.sum(ind)  # Case a = 1/|C_i|
            W[:, i] =  s2 * np.sum(Pij * f, axis=1)
        f = s1 * f + W
        z[:, :, iter] = f
    # Get the leader values at EACH layer
    leaders = []
    for iter in range(num_steps + 1):
        leader_particles = []
        for i in range(n):
            y = np.dot(z[:, i, iter], np.dot(A,z[:, :, iter]))
            ind = y == np.max(y)
            if i == np.where(ind)[0][0]:
                leader_particles.append(np.where(ind)[0][0])
        leaders.append(leader_particles)

    # Identify tokens that are not leaders but also followed (attractors)    
    attractors = np.zeros((n,num_steps+1))
    for iter in range(num_steps+1):
        for p in range(n):
            y = np.dot(z[:, p, iter], np.dot(A,z[:, :, iter]))
            attractors[p,iter] = np.argmax(y)
    unique_attractors = np.unique(attractors)
    color_index_mapping = {value: index for index, value in enumerate(unique_attractors)}

    # Set axis properties
    for ax in axs.flat:
        ax.tick_params(axis='both', which='both', length=0)  # Remove ticks
        ax.set_xticklabels([])  # Remove x-axis labels
        ax.set_yticklabels([])  # Remove y-axis labels
        ax.set_aspect('equal')  # Set equal aspect ratio
        ax.spines['top'].set_linewidth(0.5)
        ax.spines['right'].set_linewidth(0.5) 
        ax.spines['bottom'].set_linewidth(0.5)
        ax.spines['left'].set_linewidth(0.5) 
    count = 0
    for iter in saving_iter:
        axs[idx,count].hlines(0, -1.2, 1.2,colors='black',linewidth = 0.5)
        axs[idx,count].vlines(0, -1.2, 1.2,colors='black',linewidth = 0.5)
        for p in range(n):
            if tails:
                if iter > 0:
                    for t in range(0,iter):
                        x_val = z[0, p, t:t+2]
                        y_val = z[1, p, t:t+2]
                        axs[idx,count].plot(x_val, y_val, 
                            linestyle = '-',
                            color = colors[color_index_mapping[attractors[p,t]]], 
                            alpha = 1, 
                            linewidth = 0.5, 
                            zorder = 1)
            x_val1 = z[0, p, iter]
            y_val2 = z[1, p, iter]
            if p not in leaders[iter]:
                if p in attractors[:,iter]:
                    plot_color = colors[color_index_mapping[attractors[p,iter]]]
                    edge_plot_color = colors[color_index_mapping[p]]
                    axs[idx,count].scatter(x_val1, y_val2,
                                    color = plot_color,
                                    alpha=1,
                                    marker='o',
                                    linewidth=1.2,
                                    edgecolors=edge_plot_color,
                                    zorder=2,
                                    s=11)
                else:   
                    axs[idx,count].scatter(x_val1, y_val2,
                                    color = colors[color_index_mapping[attractors[p,iter]]],
                                    alpha=1,
                                    marker='o',
                                    linewidth=1.2,
                                    edgecolors=colors[color_index_mapping[attractors[p,iter]]],
                                    zorder=2,
                                    s=11)
            if p in leaders[iter]:
                    axs[idx,count].scatter(x_val1, y_val2,
                                    c = colors[color_index_mapping[attractors[p,iter]]],
                                    alpha=1,
                                    marker='*',
                                    linewidth=0.5,
                                    zorder=3,
                                    s=50)
        if iter == num_steps:
            points = np.array([[z[0, int(i), num_steps], z[1, int(i), num_steps]] for i in leaders[iter]])
            axs[idx,count].fill(points[:, 0], 
                    points[:, 1], 
                    color='blue',
                    alpha=0.1,
                    edgecolor = 'none',
                    zorder = 0)
        count = count + 1
current_datetime = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
plt.savefig(f"{dir_path}/dyn_{current_datetime}.pdf", bbox_inches="tight")
